<h1> Failed Decision Tree Attempt </h1>
Found a different tutorial on how to make a decision tree for this method. Works similar to the other method's I've found, and is easier to read and understand than the other method I ended up succeeding with. However, I could not seem to get the shape of my arrays correct to work with the decision tree. Built in datasets work from sklearn, but I've been unable to make the Kaggle datasets behave and I've run out of time on this assignment. Figured I'd include this to see if you understand where I am making a mistake. 

In [1]:
import numpy as np
from sklearn import tree

In [2]:
class Node:
    def __init__(self, gini, num_samples, num_samples_per_class, predicted_class):
        self.gini = gini
        self.num_samples = num_samples
        self.num_samples_per_class = num_samples_per_class
        self.predicted_class = predicted_class
        self.feature_index = 0
        self.threshold = 0
        self.left = None
        self.right = None

In [3]:
class DecisionTreeClassifier:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth

    def fit(self, X, y):
        self.n_classes_ = len(set(y))  
        self.n_features_ = X.shape[1]
        self.tree_ = self._grow_tree(X, y)

    def predict(self, X):
        return [self._predict(inputs) for inputs in X]

    def _gini(self, y):
        m = y.size
        return 1.0 - sum((np.sum(y == c) / m) ** 2 for c in range(self.n_classes_))

    def _best_split(self, X, y):
        # Need at least two elements to split a node.
        m = y.size
        if m <= 1:
            return None, None

        # Count of each class in the current node.
        num_parent = [np.sum(y == c) for c in range(self.n_classes_)]

        # Gini of current node.
        best_gini = 1.0 - sum((n / m) ** 2 for n in num_parent)
        best_idx, best_thr = None, None

        # Loop through all features.
        for idx in range(self.n_features_):
            # Sort data along selected feature.
            thresholds, classes = zip(*sorted(zip(X[:, idx], y)))

            num_left = [0] * self.n_classes_
            num_right = num_parent.copy()
            for i in range(1, m):  # possible split positions
                c = classes[i - 1]
                num_left[c] += 1
                num_right[c] -= 1
                gini_left = 1.0 - sum(
                    (num_left[x] / i) ** 2 for x in range(self.n_classes_)
                )
                gini_right = 1.0 - sum(
                    (num_right[x] / (m - i)) ** 2 for x in range(self.n_classes_)
                )

                gini = (i * gini_left + (m - i) * gini_right) / m

                if thresholds[i] == thresholds[i - 1]:
                    continue

                if gini < best_gini:
                    best_gini = gini
                    best_idx = idx
                    best_thr = (thresholds[i] + thresholds[i - 1]) / 2  # midpoint

        return best_idx, best_thr

    def _grow_tree(self, X, y, depth=0):

        num_samples_per_class = [np.sum(y == i) for i in range(self.n_classes_)]
        predicted_class = np.argmax(num_samples_per_class)
        node = tree.Node(
            gini=self._gini(y),
            num_samples=y.size,
            num_samples_per_class=num_samples_per_class,
            predicted_class=predicted_class,
        )

        if depth < self.max_depth:
            idx, thr = self._best_split(X, y)
            if idx is not None:
                indices_left = X[:, idx] < thr
                X_left, y_left = X[indices_left], y[indices_left]
                X_right, y_right = X[~indices_left], y[~indices_left]
                node.feature_index = idx
                node.threshold = thr
                node.left = self._grow_tree(X_left, y_left, depth + 1)
                node.right = self._grow_tree(X_right, y_right, depth + 1)
        return node

    def _predict(self, inputs):
 
        node = self.tree_
        while node.left:
            if inputs[node.feature_index] < node.threshold:
                node = node.left
            else:
                node = node.right
        return node.predicted_class

In [4]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
from sklearn import tree

with open('house-votes-84.csv', 'r') as inputFile:
    lines = inputFile.readlines()
    
    choice = 0
    X = np.array([])
    Y = np.array([])
    X_unseen = []
    Y_unseen = []

    count = 0
    for line in lines:
        if not '?' in line:

            # print(line)

            count = count + 1
            values = line.split(',')
            party = values[0]

            if 'democrat' in party:
                choice = 1
            if 'republican' in party:
                choice = 0

            instanceValues = []
            for index in enumerate(values, start=1):
                if index[0] != 1:
                    if 'n' in index[1]:
                        instanceValues.append(0)
                    if 'y' in index[1]:
                        instanceValues.append(1)

            # Take every other sample and set aside for validation of our decision tree
            # In other words, don't include these rows in our training data

            if count % 2 == 0:
                Y_unseen.append(choice)
                X_unseen.append(instanceValues)
            else:
                Y = np.append(Y,choice)
                X = np.append(X,instanceValues)

    print ('Total instances:', count)
    print ('Training instances:', len(X))
    print ('Validation instances:', len(X_unseen))

    clf = DecisionTreeClassifier()
    clf = clf.fit(X, Y)
    predictions = clf.predict(X_unseen)

    # print ("Predictions:", predictions)
    # print ("Actual vals:", Y_unseen)

    wrongPredictions = 0
    for (index, val) in enumerate(predictions):
        if Y_unseen[index] != val:
            wrongPredictions = wrongPredictions + 1
    print ('Correctly predicted %s out of %s instances' % (len(Y_unseen)
            - wrongPredictions, len(Y_unseen)))


Total instances: 233
Training instances: 1871
Validation instances: 116


IndexError: tuple index out of range